In [1]:
!pip install segmentation_models_pytorch
!pip install wandb
!pip install torcheval

!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 46.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_6

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/Dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: findit2/train_patches_masks/X51005605285_patch_7.png  
  inflating: findit2/train_patches_masks/X51006556728_patch_4.png  
  inflating: findit2/train_patches_masks/X51005447850_patch_2.png  
  inflating: findit2/train_patches_masks/X51008114321_patch_1.png  
  inflating: findit2/train_patches_masks/X51005675914_patch_2.png  
  inflating: findit2/train_patches_masks/X51006620181_patch_2.png  
  inflating: findit2/train_patches_masks/X51007846321_patch_14.png  
  inflating: findit2/train_patches_masks/X51006619343_patch_3.png  
  inflating: findit2/train_patches_masks/X51005676540_patch_1.png  
  inflating: findit2/train_patches_masks/X51005568889_patch_13.png  
  inflating: findit2/train_patches_masks/X51005442327_patch_4.png  
  inflating: findit2/train_patches_masks/X51007231331_patch_17.png  
  inflating: findit2/train_patches_masks/X51006392299_patch_12.png  
  inflating: findit2/train_patches_masks/X51006349085_patch_0

# Iniatial

In [ ]:
# !mkdir Dataset
# !mkdir Dataset/COMOFOD

# !wget -P Dataset/COMOFOD https://www.vcl.fer.hr/comofod/comofod_small.rar

In [ ]:
!unzip /content/drive/MyDrive/Dataset.zip -d /content

Archive:  /content/drive/MyDrive/Dataset.zip
 extracting: /content/findit/val_json.txt  
 extracting: /content/findit/train_json.txt  
 extracting: /content/findit/test_json.txt  
 extracting: /content/findit/findit2/val.txt  
 extracting: /content/findit/findit2/test.txt  
 extracting: /content/findit/findit2/train.txt  
 extracting: /content/findit/findit2/val_bm/X51006828201.png  
 extracting: /content/findit/findit2/val_bm/X51008114283.png  
 extracting: /content/findit/findit2/val_bm/X51005441401.png  
 extracting: /content/findit/findit2/val_bm/X51005757304.png  
 extracting: /content/findit/findit2/val_bm/X51005605333.png  
 extracting: /content/findit/findit2/val_bm/X51005711404.png  
 extracting: /content/findit/findit2/val_bm/X00016469612.png  
 extracting: /content/findit/findit2/val_bm/X51005568890.png  
 extracting: /content/findit/findit2/val_bm/X51007419197.png  
 extracting: /content/findit/findit2/val_bm/X51006414636.png  
 extracting: /content/findit/findit2/val_bm/X5

In [ ]:
# !rm -r /content/Dataset/COMOFOD/comofod_small.rar

In [ ]:
# import shutil

# shutil.rmtree('/Project/Dataset', ignore_errors=True)

# Libraries

In [1]:
# For managing COCO dataset
# from pycocotools.coco import COCO

# For creating and managing folder/ files
import glob
import os
import shutil

# For managing images
from PIL import Image
import skimage.io as io

# Basic libraries
import numpy as np
import pandas as pd
import random
import cv2

# For plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import wandb

# For importing models and working with them
## Torch
import torch
import torch.utils.data # for Dataset
import torch.nn as nn
from torch.optim import Adam
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
import segmentation_models_pytorch as smp

## Torchvision
import torchvision
from torchvision.transforms import transforms

# For creating train - test splits
from sklearn.model_selection import train_test_split

import pathlib
import pylab
import requests
from io import BytesIO
from pprint import pprint
from tqdm import tqdm
import time
from imutils import paths

# Performance Metrics
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix


# Functions - to have them separated in files
import CalcMetrics as cm
import BinaryMasks as bm
import TrainModel as trModel
import ValidateModel as valModel
import TestModel as testModel
import CreateDataset_Comofod as com
import CreateDataset_IMD2020 as imd
import CreateDataset_SROIE as sroie
import DisplayMetrics as dm
import PlotResults as pr
import MainLoop as main
import EarlyStopping as stopping


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


# %matplotlib inline

cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda


# Data

## Data Preparation

In [2]:
RESNET_SIZE = 256
transf_size = RESNET_SIZE

# Initialize transformations - train
transforms_train = transforms.Compose([
    transforms.Resize((transf_size, transf_size)),
    transforms.ToTensor(),
])

# Initialize transformations - test (optional)
transforms_test = transforms.Compose([
    transforms.Resize((transf_size, transf_size)),
    transforms.ToTensor(),
])

BATCH_SIZE = 8

In [3]:
# Train
# csv_file = '/content/train.csv'
images_dir              = '/content/findit2/train_patches'
binary_masks_dir        = '/content/findit2/train_patches_bm'
bounding_box_masks_dir  = '/content/findit2/train_patches_masks'

# Create dataset and data loader
dataset = sroie.SegmentationDataset(images_dir, binary_masks_dir, bounding_box_masks_dir, transform=transforms_train)
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [4]:
# Test
# csv_file = '/content/test.csv'
images_dir              = '/content/findit2/test_patches'
binary_masks_dir        = '/content/findit2/test_patches_bm'
bounding_box_masks_dir  = '/content/findit2/test_patches_masks'

# Create dataset and data loader
dataset = sroie.SegmentationDataset(images_dir, binary_masks_dir, bounding_box_masks_dir, transform=transforms_test)
test_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [5]:
# Val
# csv_file = '/content/val.csv'
images_dir               = '/content/findit2/val_patches'
binary_masks_dir         = '/content/findit2/val_patches_bm'
bounding_box_masks_dir   = '/content/findit2/val_patches_masks'

# Create dataset and data loader
dataset = sroie.SegmentationDataset(images_dir, binary_masks_dir, bounding_box_masks_dir, transform=transforms_test)
val_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [6]:
# original_images = [train_original_images, test_original_images, val_original_images]
# altered_images = [train_bounding_box_masks, test_bounding_box_masks, val_bounding_box_masks]
# masks = [train_binary_masks, test_binary_masks, val_binary_masks]


original_images = train_loader
altered_images = test_loader
masks = val_loader

## Main

In [ ]:
# 73c482950d37419655b401f882c34c719ca587af

# Define hyperparameters
INIT_LR = 0.0001
BATCH_SIZE = 8
NUM_EPOCHS = [10, 20, 50, 100, 150]
TEST_SPLIT = [0.1, 0.2] # 0.3
VALID_SPLIT = 0.2 # 0.1

model_type = 'GCA' # unet
channels = 3 # 6
dataset_type = 'sroie' # imd, sroie, comofod

wb_name = "GCANet_SROIE_imgs_patches"

main.main_loop(original_images, altered_images, masks, transforms_train, transforms_test,
              model_type, channels, dataset_type,
              wb_name, INIT_LR, BATCH_SIZE, NUM_EPOCHS, TEST_SPLIT, VALID_SPLIT)



wandb: Currently logged in as: bianca-apostolescu (bianca-team). Use `wandb login --relogin` to force relogin


[INFO] TEST_SPLIT = 0.1 ...
Splits, Datasets, and Dataloaders
[INFO] Total time taken to create the dataset and dataloader: 0.00s
trainSteps = 150, testSteps = 55, valSteps = 51


[INFO] Training the network for 10 epochs...


  0%|          | 0/10 [00:00<?, ?it/s]

Training...


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Validating...


TypeError: plot_results() takes 1 positional argument but 5 were given

In [8]:
!mkdir model_checkpoints